## Данные о маршрутах общественного транспорта с Единого транспортного портала

Сбор данных о маршрутах общественного транспорта с сайта [transport.mos.ru](https://transport.mos.ru/transport/schedule) (в ознакомительных целях).

In [1]:
# Устанавливаем библиотеку playwright
#pip install playwright nest_asyncio

In [2]:
# Устанавливаем браузер для работы playwright
#!playwright install

In [3]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

import pandas as pd

from datetime import datetime

nest_asyncio.apply()

Работа с данными заключается в сборе информации с веб-страниц маршрутов общественного транспорта.

Сбор данных производится в несколько этапов:

* сбор url-ссылок на маршруты;

* переход по ссылкам;

* сбор данных о наборе остановок рейсов маршрута.

In [587]:
# Создаём пустой список, в который будут записываться ссылки маршрутов с сайта
links_routes = []

# Создаём переменную для записи даты и времени получения данных
timestamp = datetime.now().strftime("%d.%m.%Y %H:%M")

# Создаём функцию для скроллинга страницы до конца, чтобы иметь возможность загрузить ссылки на все маршруты
async def scroll_to_bottom(page):
    previous_height = await page.evaluate("document.body.scrollHeight")
    
    while True:
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        
        await page.wait_for_timeout(1000)
        new_height = await page.evaluate("document.body.scrollHeight")
        if new_height == previous_height:
            break
        
        previous_height = new_height

# Создаём основную функцию, где будет производиться сбор ссылок на страницы с маршрутами
async def run_playwright():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        # переходим по ссылке с списком маршрутов
        await page.goto("https://transport.mos.ru/transport/schedule")
        
        # Обращаемся к функции для скроллинга страницы
        await scroll_to_bottom(page)
        
        # Ожидаем прогрузки на странице данных в селекторе ссылок 
        await page.wait_for_selector("a")

        # Получаем данные из тега <a class="ts-row ">
        data = await page.evaluate('''() => {
            return Array.from(document.querySelectorAll('a.ts-row ')).map(a => a.href);
        }''')
        
        # Поэлементно добавляем данные, полученные выше, в список
        for item in data:
            links_routes.append(item)

        await browser.close()
        
import asyncio
asyncio.run(run_playwright())

python3.9(58540) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [589]:
# Выводим количество элементов в списке
len(links_routes)

1117

In [579]:
# Создаём функцию для сбора данных со страницы маршрута
async def data_page(page, radio_count):
    global data_list # объявление глобальной переменной, чтобы она могла быть использована в любой части программы
    # Пробуем достать данные из кода страницы
    try:
        # <i class ="..."> - тип маршрута (внутри тега)
        h1_element = await page.query_selector("h1.h3mb")
        i_elements = await h1_element.query_selector_all("i")
        class_name = [await i.evaluate('el => el.className') for i in i_elements]
        print(f'1 - {class_name}')
        try:
            # <h1 class ="h3mb"> - номер маршрута (текст в теге)
            h1_text = await h1_element.inner_text()
        except:
            h1_text = "-"
    except:
        class_name = "-"
    
    try:
        # <span id="schedule-route-header"> - название рейса (текст внутри "span")
        span_header_element = await page.query_selector("span#schedule-route-header")
        try:
            span_header_text = await span_header_element.inner_text()
        except:
            span_header_text = "-"
    except:
        span_header_text = ""
    
    try:
        # <div id="schedule-table"> - остановочные пункты
        stops_elements = await page.query_selector_all("div#schedule-table")
        stops = [await d.inner_text() for d in stops_elements]
        print(f'4 - {stops}')
        print("----------")
    except:
        stops = "-"
    
    # Добавляем данные, полученные выше, в список
    data_list.append([class_name, h1_text, span_header_text, stops, radio_count])
    
    # Возвращаем список с данными
    return data_list

In [604]:
# Создаём пустой список, куда будут добавляться данные, собранные с веб-страницы
data_list = []

# Создаём функцию для проверки наличия кнопок переключения рейсов
async def check_div_availability():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        
        # Переходим по собранным ссылкам 
        for link in links_routes:
            await page.goto(link)

            await page.wait_for_timeout(2000)
            #result = await data_page(page)

            # Собираем все радиокнопки (рейсы в одном направлении) на странице с помощью xpath селектора        
            try:
                radios = await page.query_selector_all("//div[@id='mgt_schedule_trip']//span[@class='sp_all']")
                print(f"Количество кнопок {len(radios)}")
                radio_count = len(radios)
                for radio in radios:
                    try:
                        await radio.check() # активируем радиокнопку
                        await page.wait_for_timeout(2000)
                        #result = await data_page(page, radio_count)
                    except:
                        continue
            except:
                continue
            result = await data_page(page, radio_count)

            # Проверяем наличие тега <i class="i.ic.ic-change-a-b"> для обнаружения рейсов обратнгом направления
            try:
                button_direction = await page.query_selector("i.ic.ic-change-a-b")
                # Кликаем на кнопку смены направления
                await button_direction.click()
                await page.wait_for_timeout(2000)
                
                try:
                    # Собираем все радиокнопки с помощью xpath селектора        
                    radios = await page.query_selector_all("//div[@id='mgt_schedule_trip']//span[@class='sp_all']")
                    print(f"Количество кнопок {len(radios)}")
                    radio_count = len(radios)
                    
                    for radio in radios:
                        try:
                            await radio.check() # активируем радиокнопку
                            await page.wait_for_timeout(2000)
                            result = await data_page(page, radio_count)
                        except:
                            continue

                except:
                    continue
            except:
                continue
            
            # Передаём полученные данные для дальнейшего сбора информации с помощью функции "data_page"
            result = await data_page(page, radio_count)
            
            await page.wait_for_timeout(2000)
            
        await browser.close()
        
asyncio.run(check_div_availability())

python3.9(61288) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Количество кнопок 2
1 - ['ic ic-xl icon-tramway']
4 - ['МЦД Калитники\nОктябрьское трамвайное депо\nМ. Калитниковский пр.\nВоловья ул.\nБ. Калитниковская ул.\nК/т "Победа" - Детский театр\nМетро "Пролетарская"\nДинамовская улица\nНовоспасский мост\nКожевническая улица\nПавелецкий вокзал\nПавелецкий вокзал\nМетро "Павелецкая"\nВишняковский переулок\nМетро "Третьяковская"\nМетро "Новокузнецкая"\nКомиссариатский мост\nЯузские ворота\nВоронцово Поле\nКазарменный пер.\nПокровские Ворота\nБ. Харитоньевский пер.\nМетро "Чистые пруды"\nБ. Харитоньевский пер.\nПокровские Ворота\nКазарменный пер.\nВоронцово Поле\nЯузские ворота\nКомиссариатский мост\nМетро "Новокузнецкая"\nМетро "Третьяковская"\nВишняковский переулок\nМетро "Павелецкая"\nПавелецкий вокзал\nКожевническая улица\nНовоспасский мост\nДинамовская улица\nМетро "Пролетарская"\nКинотеатр "Победа" - Детский театр\nАбельмановская Застава\nБ. Калитниковская ул.\nВоловья ул.\nМалый Калитниковский проезд']
----------
Количество кнопок 0
1 - [

Количество кнопок 0
1 - ['ic ic-xl ic-bus']
4 - ['Парк "Фили"\nУл. Барклая\nБ. Филевская ул., 23\n51-я гор. б-ца\nМинская ул.\nМетро "Филёвский парк"\nПлощадь Ромена Роллана\nПарк Победы\nМетро "Минская"\nКаменная плотина\nМосфильмовский пруд\nМосфильмовская ул.\nМетро "Ломоносовский проспект"\nМенделеевская ул.\nБиблиотека МГУ\nУлица Лебедева\nМетро "Университет"\nМетро "Университет"\nДрамтеатр\nЛенинский проспект\nЦентр здоровья детей\nЧерёмушкинский рынок\nУл. Ивана Бабушкина\nМетро "Профсоюзная"\nМетро "Профсоюзная"\nНовочеремушкинская ул.\nМФЦ Черёмушки\n23-й квартал Новых Черёмушек']
----------


TimeoutError: Timeout 30000ms exceeded.

In [605]:
len(data_list)

20

In [599]:
data_list

[[['ic ic-xl icon-tramway'],
  ' А',
  'МЦД Калитники - Октябрьское трамвайное депо',
  ['МЦД Калитники\nОктябрьское трамвайное депо\nМ. Калитниковский пр.\nВоловья ул.\nБ. Калитниковская ул.\nК/т "Победа" - Детский театр\nМетро "Пролетарская"\nДинамовская улица\nНовоспасский мост\nКожевническая улица\nПавелецкий вокзал\nПавелецкий вокзал\nМетро "Павелецкая"\nВишняковский переулок\nМетро "Третьяковская"\nМетро "Новокузнецкая"\nКомиссариатский мост\nЯузские ворота\nВоронцово Поле\nКазарменный пер.\nПокровские Ворота\nБ. Харитоньевский пер.\nМетро "Чистые пруды"\nБ. Харитоньевский пер.\nПокровские Ворота\nКазарменный пер.\nВоронцово Поле\nЯузские ворота\nКомиссариатский мост\nМетро "Новокузнецкая"\nМетро "Третьяковская"\nВишняковский переулок\nМетро "Павелецкая"\nПавелецкий вокзал\nКожевническая улица\nНовоспасский мост\nДинамовская улица\nМетро "Пролетарская"\nКинотеатр "Победа" - Детский театр\nАбельмановская Застава\nБ. Калитниковская ул.\nВоловья ул.\nМалый Калитниковский проезд\nОкт